# Related Anime Graph
* This is a simple signal based off of related series
* Two series are said to be related if one is a sequel/prequel/etc. of the other
* The predicted score of a series is the average of all related series

In [1]:
# CHANGE THIS PARAMETER
recommendee = "taapaye"

In [2]:
import os
import pickle

import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
# TODO read these hyperparameters from a file
# Change this to get recommendations for a different user
signal_name = f"related"

In [4]:
outdir = f"../../data/recommendations/{recommendee}"
os.chdir(outdir)

In [5]:
user_df = pickle.load(open("user_anime_list.pkl", "rb"))

In [6]:
#related_df = pickle.load(open("../../processed_data/related_anime_graph.pkl", "rb"))

In [7]:
def get_oos_prediction(user_df, related_df, anime_id):
    user_oos = user_df.loc[lambda x: x["anime_id"] != anime_id]
    series_df = (
        user_oos.merge(related_df, on="anime_id")
        .groupby("series_id")[["score", "score_var"]]
        .mean()
    )
    pred_scores = related_df.merge(series_df, on="series_id", how="left").fillna(0)
    pred_scores = pred_scores.rename(
        {"score": "delta", "score_var": "delta_var"}, axis=1
    ).drop("series_id", axis=1)
    return pred_scores.loc[lambda x: x["anime_id"] == anime_id]

In [8]:
def store_signals(signal_name):
    related_df = pickle.load(open(f"../../processed_data/{signal_name}_anime_graph.pkl", "rb"))
    signal_name=signal_name.replace('_', '')
    
    loocv_dfs = []
    for anime_id in tqdm(user_df["anime_id"]):
        loocv_dfs.append(get_oos_prediction(user_df, related_df, anime_id))
    loocv_df = pd.concat(loocv_dfs, ignore_index=True).set_index("anime_id")
    loocv_df.to_pickle(f"{signal_name}_loocv.pkl")
    
    pred_scores = (
        user_df.merge(related_df, on="anime_id").groupby("series_id")["score"].mean()
    )
    pred_vars = (
        user_df.merge(related_df, on="anime_id")
        .groupby("series_id")["score_var"]
        .apply(lambda x: x.sum() / x.size ** 2)
    )
    signal = pd.DataFrame()
    signal["delta"] = pred_scores
    signal["delta_var"] = pred_vars
    signal = (
        signal.merge(related_df, on="series_id")
        .drop("series_id", axis=1)
        .set_index("anime_id")
    )
    signal.to_pickle(f"{signal_name}.pkl")

In [9]:
store_signals('strict_relations')
store_signals('weak_relations')
store_signals('all_relations')

100%|████████████████████████████████████████████████████████████████████████████████| 425/425 [00:04<00:00, 96.20it/s]
